In [ ]:

import numpy as np
import plotly.io as pio
from tvDatafeed import Interval , TvDatafeed
import plotly.graph_objects as go
import pandas as pd

from tvdataclass.scratch import holdcandle

pio.templates

username = 'Fook_Yew'
password = 'Tr4d1ng1sc00l'
tv = TvDatafeed(username, password)


 ##########################################

class TestCandle:

    def __init__(self, interval: Interval, n_bars: int) -> object:

        self.interval = interval
        self.n_bars:int = n_bars
        self.name = self.set_name()
        self.df = self.set_df()
        self.long_tester = self.get_long_tester()
        self.short_tester = self.get_short_tester()


    def set_name(self) -> str:
        return str(self.interval).split('.in_')[-1]

    def set_df(self) -> pd.DataFrame:
        return tv.get_hist(symbol='BTCUSDT.P', exchange='zoomex', interval=self.interval, n_bars=self.n_bars)

    def get_long_tester(self) -> pd.DataFrame:
        return self.df.low

    def get_short_tester(self)->pd.DataFrame:
        return self.df.high

############################################################################################################################################

class HoldCandle:

    def __init__(self, interval, n_bars):

        self.interval = interval
        self.n_bars:int = n_bars
        self.name = self.set_name()

        self.df = self.set_df()
        self.get_hold_indexs()
        self.longs = self.get_longs()
        self.shorts= self.get_shorts()
    def set_name(self)->str:

        return str(self.interval).split('.in_')[-1]
    def set_df(self)->pd.DataFrame:

        return tv.get_hist(symbol='BTCUSDT.P', exchange='zoomex', interval=self.interval, n_bars=self.n_bars)
    def get_hold_indexs(self)->None:

        self.short_indexes = []
        self.long_indexes = []
        self.df['up_down'] = np.sign(self.df.close - self.df.open)

        for time in self.df.index[:-4]:
            sl = slice(time, time + 3 * (self.df.index[1] - self.df.index[0]))
            chunk = self.df.loc[sl]

            if all(chunk['up_down'] == [1, 1, -1, -1]):
                self.short_indexes.extend(chunk.index[:2])

            elif all(chunk['up_down'] == [-1, -1, 1, 1]):
                self.long_indexes.extend(chunk.index[:2])

    def get_longs(self)->pd.DataFrame:

        return pd.concat([ self.df.loc[self.long_indexes].high,  self.df.loc[self.long_indexes].open]).sort_index()
    def get_shorts(self)->pd.DataFrame:

        return pd.concat([self.df.loc[self.short_indexes].low, self.df.loc[self.short_indexes].open]).sort_index()

####################################################################################################################

In [204]:
########################


class Untested(HoldCandle, TestCandle):
    """
    will allow to get any properties from parent hold or test but must pass as args
    """
    def __init__(self, holdcandle, testcandle) -> object:

        self.holdcandle =holdcandle
        self.testcandle =testcandle
        self.name = self.holdcandle.name
        self.name_test=self.testcandle.name

        self.longs = self.test_longs()
        self.shorts = self.test_shorts()

    def test_hold(self, hold, test) -> pd.DataFrame:
        self.hold = hold
        self.test = test

        print(f' To Test: {len(self.hold)}')

        #return true where tester is in
        mask = round(self.hold).isin(round(self.test))
        ix_drop = self.hold[mask].index

        print(f' Tested: {len(ix_drop)}')
        return self.hold.drop(labels=ix_drop)

    def test_longs(self):
        print('LONGS')
        return self.test_hold(self.holdcandle.longs, self.testcandle.long_tester)

    def test_shorts(self):
        print('SHORTS')
        
        return self.test_hold(self.holdcandle.shorts, self.testcandle.short_tester)

############################################################################################################################################


In [205]:

import plotly.graph_objects as go
import pandas as pd

import matplotlib.pyplot as plt
from functools import wraps


###################################################################################################################################################
###################################################################################################################################################

class PlotCandle(HoldCandle):

    def __init__(self, holdcandle, testcandle):
        self.hc=holdcandle
        self.tc = testcandle
        self.df = self.hc.df
        self.name = self.hc.name

        self.long_hlines = self.get_hlines(self.tc.longs)
        self.short_hlines = self.get_hlines(self.tc.shorts)
       

        # self.fig_holds = self.plot2()

    def get_hlines(self, hold)->pd.DataFrame:
        hlines = pd.DataFrame(hold)
        hlines['x1'] = self.df.index[-1]
        hlines['y1'] = hlines[0]
        hlines.index.name='x0'
        hlines = hlines.reset_index()
        return hlines.rename(columns={0:'y0'})


    def plot(self):
        

        fig = go.Figure(data=[go.Candlestick(x=self.df.index, open=self.df['open'], high=self.df['high'], low=self.df['low'], close=self.df['close'])], layout=go.Layout())
        fig.update_layout(title=self.name, xaxis_rangeslider_visible=False, xaxis_title='Date', yaxis_title='Price', template="plotly_dark")
        
        fig.update_traces(increasing_line_color='white', selector=dict(type='candlestick'))
        fig.update_traces(decreasing_line_color='blue', selector=dict(type='candlestick'))
        return fig
    


    def add_lines(self, hlines, fig, color):

        for row in hlines.index:
            x0, y0, x1, y1 = hlines.loc[row].values
            fig.add_shape(type='line', line=dict(color=color), label=dict(text=y1, textposition='end', padding=0, font=dict(size=10), xanchor='right', yanchor='middle'), x0=x0, y0=y0, x1=x1, y1=y1)
        return fig


In [206]:


INTERVALS = ['in_1_minute', 'in_3_minute', 'in_5_minute', 'in_15_minute', 'in_1_hour', 'in_4_hour', 'in_daily']
COLORS = ['grey', 'pink', 'green', 'orange', 'blue', 'yellow', 'red']
N_BARS = [1000, 300, 200, 100, 40, 20, 10]
MINS = [1, 3, 5, 15, 60, 240, 1440]


d = {'int':INTERVALS,'n': N_BARS, 'c': COLORS, 'mins':MINS}
df = pd.DataFrame.from_dict(d)
df.index = df['int']


In [209]:
pc = PlotCandle(hc, tc)

AttributeError: 'TestCandle' object has no attribute 'longs'

In [200]:
pc.short_hlines['interval'] = 'dssd'

In [202]:
pc.long_hlines

,x0,y0,x1,y1,interval
0,2024-10-19 01:40:00,68346.8,2024-10-19 18:00:00,68346.8,dssd
1,2024-10-19 01:40:00,68337.8,2024-10-19 18:00:00,68337.8,dssd
2,2024-10-19 01:45:00,68318.5,2024-10-19 18:00:00,68318.5,dssd
3,2024-10-19 01:45:00,68318.4,2024-10-19 18:00:00,68318.4,dssd
4,2024-10-19 02:55:00,68403.5,2024-10-19 18:00:00,68403.5,dssd
5,2024-10-19 02:55:00,68401.7,2024-10-19 18:00:00,68401.7,dssd
6,2024-10-19 03:00:00,68401.6,2024-10-19 18:00:00,68401.6,dssd
7,2024-10-19 03:00:00,68401.6,2024-10-19 18:00:00,68401.6,dssd
8,2024-10-19 04:45:00,68287.5,2024-10-19 18:00:00,68287.5,dssd
9,2024-10-19 04:45:00,68287.4,2024-10-19 18:00:00,68287.4,dssd


In [208]:
hcn = 3
hc = HoldCandle(Interval.in_daily, hcn)
hcmins = int(hcn*24*60)

tc = TestCandle(Interval.in_1_minute, hcmins)


for i in INTERVALS[1:3]:

    n = int(round(hcmins / df.mins.loc[i]))
    if ntc > 5000:
        ntc = 5000
    hc = HoldCandle(Interval[i], n)
    pc = Untested(hc, tc)
    longs = ut.longs

LONGS
 To Test: 352
 Tested: 294
SHORTS
 To Test: 364
 Tested: 319
LONGS
 To Test: 216
 Tested: 194
SHORTS
 To Test: 196
 Tested: 178


In [150]:
tc = HoldCandle(Interval.in_5_minute, 200)

In [176]:
class Plot2(PlotCandle):
    def __init__(self, holdcandle, testcandle, plot_kws=[], **kwargs):
        super().__init__(holdcandle, testcandle)
        self.plot_kws = plot_kws
       
        for k in kwargs.keys():
            self.__setattr__(k, kwargs[k])
            self.plot_kws.append(k)
       
        self.fig=self.plot()
        
    def modplot(self):
        for kw in self.plot_kws:
            print(kw, self.__dict__[kw])
            self.fig.layout[kw] = self.__dict__[kw]

    def plot_longs(self):
        self.add_lines(self.long_hlines, self.fig, color='green')
        
    def plot_shorts(self):
        self.add_lines(self.short_hlines, self.fig, color='red')
        
    def plot_both(self):
        self.plot_longs()
        self.plot_shorts()
        
    
        



In [ ]:
def add_multi_holds():
    

In [197]:
pc2 = Plot2(hc, tc)
pc2.plot_both()
pc2.fig.show()